In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from snmfem.experiments import load_samples, print_results, load_data, run_experiment
from snmfem.measures import KL, trace_xtLx
from snmfem.updates import multiplicative_step_a, multiplicative_step_p, multiplicative_step_aq
from snmfem.conf import log_shift, dicotomy_tol
from snmfem.laplacian import sigmaL, create_laplacian_matrix


In [ ]:
def create_toy_problem(l = 25, k = 3, p = 100, c = 10, n_poisson=200, force_simplex=True):

    A = np.random.rand(k,p)
    if force_simplex:
        A = A/np.sum(A, axis=0, keepdims=True)
    
    G = np.random.rand(l,c)
    P = np.random.rand(c,k)
    GP = G @ P

    X = GP @ A

    Xdot = 1/n_poisson * np.random.poisson(n_poisson * X)

    return G, P, A, X, Xdot

In [ ]:
def smooth_l2_surrogate(At, A=None, sigmaL=sigmaL, lambda_L=1):
    tmp = (At @ L)
    t1 = np.sum(At * tmp)
    if not(A is None):
        t2 = np.sum(A * tmp)
        t3 = np.sum((At-A)**2)
    else:
        t2 = t1
        t3 = 0
    return lambda_L / 2 * (2*t2 -t1 + sigmaL * t3)

def diff_surrogate(At, A, sigmaL=sigmaL, lambda_L=1):
    b_inf = trace_xtLx(L, A.T) * lambda_L / 2
    b_supp = smooth_l2_surrogate(At, A=A, sigmaL=sigmaL, lambda_L=lambda_L)
    return b_supp - b_inf
    
L = create_laplacian_matrix(4, 3)
for i in range(10):
    A1 = np.random.randn(3, 12)
    v1 = smooth_l2_surrogate(A1)
    v2 = smooth_l2_surrogate(A1, A1)
    v3 = trace_xtLx(L, A1.T) / 2
    np.testing.assert_almost_equal(v1, v2)
    np.testing.assert_almost_equal(v1, v3)

    for j in range(10):
        A2 = np.random.randn(3, 12)
        v4 = smooth_l2_surrogate(A1, A2)
        v5 = trace_xtLx(L, A2.T) / 2
        assert v4 >= v5
        d = diff_surrogate(A1, A2, sigmaL=sigmaL)
        np.testing.assert_almost_equal(v4 - v5 , d)

    
    

In [ ]:

shape_2d = [10, 15]
k = 5
n_poisson = 200
G, P, A, Xtrue, X = create_toy_problem(p = shape_2d[0]*shape_2d[1], k=k, n_poisson=n_poisson)

P1 = np.random.rand(*P.shape)
A1 = np.random.rand(*A.shape)
P2 = P1.copy()
A2 = A1.copy()

P3 = P1.copy()
A3 = A1.copy()
P0 = P1.copy()
A0 = A1.copy()

L = create_laplacian_matrix(*shape_2d)

lambda_L = 10

def loss(P, A):
    DA = G @ P @ A
    v1 = KL(X, DA) 
    v2 = trace_xtLx(L, A.T)
    return v1 + lambda_L/2 * v2

maxit = 100
force_simplex = False
loss0 = [loss(P0, A0)]

loss1 = [loss(P1, A1)]
loss2 = [loss(P2, A2)]
diff_loss2 = []
loss3 = [loss(P3, A3)]
diff_loss3 = []

# gamma2 = sigmaL
gamma2 = 0.2
gamma3 = sigmaL
# gamma3 = 0.005
gamma3_vec = [gamma3]
d3 = []

for i in range(maxit):
    A0 = multiplicative_step_a(X, G, P0, A0, force_simplex=force_simplex, lambda_L=lambda_L, L=L)
    P0 = multiplicative_step_p(X, G, P0, A0)
    loss0.append(loss(P0, A0))
    
    A1 = multiplicative_step_aq(X, G, P1, A1, force_simplex=force_simplex, lambda_L=lambda_L, L=L)
    P1 = multiplicative_step_p(X, G, P1, A1)
    loss1.append(loss(P1, A1))

    
    A2 = multiplicative_step_aq(X, G, P2, A2, force_simplex=force_simplex, lambda_L=lambda_L, L=L, sigmaL=gamma2)
    diff_loss2.append(loss2[-1]-loss(P2, A2))

    P2 = multiplicative_step_p(X, G, P2, A2)
    loss2.append(loss(P2, A2))
    A3_old = A3.copy()
    A3 = multiplicative_step_aq(X, G, P3, A3, force_simplex=force_simplex, lambda_L=lambda_L, L=L, sigmaL=gamma3)
    d3.append(diff_surrogate(A3_old, A3, sigmaL=gamma3))
    diff_loss3.append(d3)
    if d3[-1]>0:
        gamma3 = gamma3 / 1.2
    else:
        gamma3 = gamma3 * 1.5
    gamma3_vec.append(gamma3)
        
    P3 = multiplicative_step_p(X, G, P3, A3)
    loss3.append(loss(P3, A3))
    

In [ ]:
gamma3

In [ ]:
for i in range(10*maxit):
    A2 = multiplicative_step_aq(X, G, P2, A2, force_simplex=force_simplex, lambda_L=lambda_L, L=L, sigmaL=gamma2)
    P2 = multiplicative_step_p(X, G, P2, A2)

l_infty = loss(P2, A2)

In [ ]:
iterations = np.arange(maxit+1)+1
plt.plot(iterations, np.array(loss0)-l_infty, "kx-", label="KL surrogate")
plt.plot(iterations, np.array(loss1)-l_infty, "bx-", label="L2 surrogate - theoretical step")
plt.plot(iterations, np.array(loss2)-l_infty, "rx-", label="L2 surrogate - optimal fixed step")
plt.plot(iterations, np.array(loss3)-l_infty, "gx-", label="L2 surrogate - linesearch")
plt.yscale("log")
plt.xscale("log")
plt.legend()


In [ ]:
plt.plot(gamma3_vec, ".-")
plt.yscale("log")
plt.title("Evolution of the step size")


In [ ]:
gamma3

# Dicotomy problem

In [ ]:
# def f(x, a, b, c):
#     v = x + b
#     return v - np.sqrt( v**2 + 4*a*c) +2*a 
# p = 1
# x = np.arange(-2,2, 0.1)
# a = np.random.rand(p)
# a = 0.5
# b = np.random.rand(p)
# c = np.random.rand(p)
# plt.plot(x, f(x,a,b,c))
# plt.plot(x, np.zeros_like(x))